In [3]:
pip install tensorflow

  Using cached tensorflow-2.8.0-cp38-cp38-win_amd64.whl (438.0 MB)

  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached grpcio-1.44.0-cp38-cp38-win_amd64.whl (3.4 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached tensorflow_io_gcs_filesystem-0.24.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Processing c:\users\mollie\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501\termcolor-1.1.0-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using c

In [4]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [6]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 50

In [7]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [8]:
df = pd.read_csv('normalised_clean_data.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]




In [15]:
for i in [16]:
  ENCODING_DIM = i
  auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
  q = encode.predict(X)
  w = np.array(q)
  df = pd.DataFrame(data=w,index=None,columns=None)
  df['label'] = y
  name = "data " + str(i)
  df.to_csv('data16.csv', index=False)

Epoch 1/50
469/469 [==============================] - 2s 3ms/step - loss: 0.0083 - rms: 0.0645 - mae: 0.0271
Epoch 2/50
469/469 [==============================] - 1s 3ms/step - loss: 9.2416e-04 - rms: 0.0304 - mae: 0.0167
Epoch 3/50
469/469 [==============================] - 1s 3ms/step - loss: 5.2490e-04 - rms: 0.0229 - mae: 0.0110
Epoch 4/50
469/469 [==============================] - 1s 3ms/step - loss: 4.3876e-04 - rms: 0.0209 - mae: 0.0099
Epoch 5/50
469/469 [==============================] - 1s 3ms/step - loss: 3.5741e-04 - rms: 0.0189 - mae: 0.0091
Epoch 6/50
469/469 [==============================] - 1s 3ms/step - loss: 2.8780e-04 - rms: 0.0170 - mae: 0.0081
Epoch 7/50
469/469 [==============================] - 1s 3ms/step - loss: 2.5099e-04 - rms: 0.0158 - mae: 0.0072
Epoch 8/50
469/469 [==============================] - 1s 3ms/step - loss: 2.3548e-04 - rms: 0.0153 - mae: 0.0069
Epoch 9/50
469/469 [==============================] - 1s 3ms/step - loss: 2.2614e-04 - rms: 0.0150 -

In [16]:
df = pd.read_csv('data16.csv')

In [17]:
df_train, df_test_valid = train_test_split( df ,stratify=df['label'], test_size=0.3)
df_test, df_validation = train_test_split( df_test_valid, stratify=df_test_valid['label'], test_size=0.5)

In [18]:
df_train.to_csv('training_set_autoencoded.csv', index=False)
df_validation.to_csv('validation_set_autoencoded.csv', index=False)
df_test.to_csv('test_set_autoencoded.csv', index=False)